In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io
from skimage import morphology
from skimage.measure import label, regionprops

# Microfluidics

In [ ]:
# USER INPUT
input_dir = "path/to/macro/output/directory/""
output_dir = "/where/to/save/csv/files/"
# END OF USER INPUT

In [ ]:
prefix_hough = "HoughLabels_DriftCorrected_"
prefix_cells = "SegmentedCells_DriftCorrected_"
prefix_all = "DriftCorrected_"
ext = ".tif"

In [ ]:
def find_files(input_dir):
    all_files = os.listdir(input_dir)
    filelist = []
    for file in all_files:
        if file.startswith(prefix_hough) and file.endswith(ext):
            filelist.append(file)
    return filelist

In [ ]:
def read_in_files(input_dir, file, prefix_hough, prefix_cells):
    filename_hough = os.path.join(input_dir, file)
    filename_cells = os.path.join(input_dir, file.replace(prefix_hough, prefix_cells))
    hough = io.imread(filename_hough)
    cells = io.imread(filename_cells)
    return hough, cells

In [ ]:
def create_mask(n, hough, dilate=0, erode=0):
    mask = hough == n
    disk = morphology.disk(dilate)
    mask = morphology.binary_dilation(mask, selem=disk)
    mask = morphology.binary_erosion(mask, selem=disk)
    return mask

In [ ]:
def count_all_cells(cells, n, title, time_axis=0):
    cells = np.moveaxis(cells, time_axis, 0)
    cols = ['File', 'Mask_number', 'Frame', 'Number_of_cells']
    df = pd.DataFrame(columns=cols)
    for i, frame in enumerate(cells):
        labelled_cells = label(frame)
        content = {'File': title,
                   'Mask_number': n + 1,
                   'Frame': i,
                   'Number_of_cells': np.max(labelled_cells)}
        df = df.append(content, ignore_index=True)
    return df

In [ ]:
def count_cells(cells, mask, n, title, time_axis=0):
    masked_cells = cells * mask
    masked_cells = np.moveaxis(masked_cells, time_axis, 0)
    # total cell count at frame 0
    cells = np.moveaxis(cells, time_axis, 0)
    total_initial_cells = np.max(label(cells[0, ...]))
    # cells in microfluidic wells over time
    cols = ['File', 'Mask_number', 'Frame', 'Number_of_cells', 'Total_initial_number_of_cells']
    df = pd.DataFrame(columns=cols)
    for i, frame in enumerate(masked_cells):
        labelled_cells = label(frame)
        content = {'File': title,
                   'Mask_number': n,
                   'Frame': i,
                   'Number_of_cells': np.max(labelled_cells),
                   'Total_initial_number_of_cells': total_initial_cells}
        df = df.append(content, ignore_index=True)
    return df

In [ ]:
def measure_intensity(image, mask, n, title, time_axis=0):
    masked_iamge = image * mask
    masked_iamge = np.moveaxis(image, time_axis, 0)
    cols = ['File', 'Mask_number', 'Frame',
            'Green_intensity_mean', 'Green_intensity_stdev', 
            'Green_intensity_min', 'Green_intensity_max',
            'Red_intensity_mean',  'Red_intensity_stdev', 
            'Red_intensity_min',  'Red_intensity_max']
    df = pd.DataFrame(columns=cols)
    for i, frame in enumerate(masked_iamge):
        red_channel = image[3, ...]
        green_channel = image[2, ...]
        content = {'File': title,
                   'Mask_number': n,
                   'Frame': i,
                   'Green_intensity_mean': np.mean(green_channel[mask]),
                   'Green_intensity_stdev': np.std(green_channel[mask]),
                   'Green_intensity_min': np.min(green_channel[mask]),
                   'Green_intensity_max': np.max(green_channel[mask]),
                   'Red_intensity_mean': np.mean(red_channel[mask]),
                   'Red_intensity_stdev': np.std(red_channel[mask]),
                   'Red_intensity_min': np.min(red_channel[mask]),
                   'Red_intensity_max': np.max(red_channel[mask])}
        df = df.append(content, ignore_index=True)
    return df

In [ ]:
filelist = find_files(input_dir)
output_filename = os.path.join(output_dir, "JenniferPayne_MicrofluidicAntibiotics.csv")
print(len(filelist))
print(output_filename)

In [ ]:
# count cell numbers
output_cellnumbers_filename = os.path.join(csv_dir, "Intensity_JenniferPayne_MicrofluidicAntibiotics.csv")
big_df = pd.DataFrame()
for file in filelist:
    print(file)
    title = file.replace(prefix_hough, "")
    hough, cells = read_in_files(input_dir, file, prefix_hough, prefix_cells)
    n_masks = np.max(hough)
    for n in range(n_masks):
        print(f"Mask {n + 1}")
        mask = create_mask(n, hough, dilate=20)
        df = count_cells(cells, mask, n, title)
        big_df = pd.concat([big_df, df])
        # perhaps add a check to see if you're overwriting a file
        big_df.to_csv(output_cellnumbers_filename)

print("Finished!")

In [ ]:
# measure fluorescence intensity
output_fluorescence_filename = os.path.join(csv_dir, "Intensity_JenniferPayne_MicrofluidicAntibiotics.csv")
big_df = pd.DataFrame()
for file in filelist:
    print(file)
    title = file.replace(prefix_hough, "")
    hough, image = read_in_files(input_dir, file, prefix_hough, prefix_all)
    n_masks = np.max(hough)
    for n in range(1, n_masks + 1):
        print(f"Mask {n}")
        mask = create_mask(n, hough, erode=10)
        df = measure_intensity(image, mask, n, title)
        # write out df?
        big_df = pd.concat([big_df, df])
        # check if you're overwriting your own file
        big_df.to_csv(output_fluorescence_filename)

print("Finished!")